In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
sys.path.append('..')
from PIL import Image
from tqdm import trange
import gymnasium as gym
import numpy as np
from sapien.core import Pose
import matplotlib.pyplot as plt
import os
import warnings
import shutil
import random
import math
from plot_utils import *
from itertools import product, combinations, permutations
#!wget https://dl.fbaipublicfiles.com/habitat/ReplicaCAD/hab2_bench_assets.zip -P data
#!cd data && unzip -q hab2_bench_assets.zip -d hab2_bench_assets
from utils import *
from agent_env_utils import *
from config import (
    data_dir, initial_dir, final_dir,
    size_types, space_types, bound_types,
    moving_directions, color_maps,
    potential_sizes, direction_length, scales,
    background_quadrants, directions
)
from scene_helpers import get_position_range, create_config, create_directories, finalize_quadrant

In [2]:
def place_main_objects(env, configs):
    base_size = 0.05
    main_placements = [(i + 1, i, "right") for i in range(len(configs) - 1)]
    distance = np.random.uniform(3 * base_size, 3.5 * base_size) * np.sqrt(2)
    
    for direction_config in main_placements:
        move_obj, ref_obj, direction = direction_config
        
        # Calculate new distance for the object to be placed
        if direction == "top" and configs[move_obj]['obj_type'] == "custom":
            new_distance = distance + 2 * base_size 
        else:
            new_distance = distance
        
        # Spawn the moving object
        env.spawn_next(direction_config, distance=new_distance)

        # Adjust the Z position for the moving object
        z_offset_move = configs[move_obj]['size'] *0.2
        current_pose_move = env.objects[move_obj].get_pose()  
        new_z_position_move = current_pose_move.p[2] + z_offset_move
        env.objects[move_obj].set_pose(Pose([current_pose_move.p[0], current_pose_move.p[1], new_z_position_move]))

        # Adjust the Z position for the reference object
        z_offset_ref = configs[ref_obj]['size'] * 0.2
        current_pose_ref = env.objects[ref_obj].get_pose()  
        new_z_position_ref = current_pose_ref.p[2] + z_offset_ref
        env.objects[ref_obj].set_pose(Pose([current_pose_ref.p[0], current_pose_ref.p[1], new_z_position_ref]))



In [3]:
# Define directories
data_dir = "../benchmark/visual/color"
final_dir = create_directories(data_dir, empty=True)

# Initialize pair of colors
num_object = 2
color_keys = color_maps.keys()
color_pairs = list(itertools.permutations(color_keys, num_object))
random.seed(42)
random.shuffle(color_pairs)

object_shapes = ["sphere", "cube"]
object_size_ranges = {
    "sphere": (0.03, 0.07),
    "cube": (0.05, 0.08),
    "random_sphere": (0.01, 0.05)
}

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    env = gym.make("CustomEnv-v0", obs_mode="rgbd")

    outputs = []
    idx = 0
    for background in trange(4):
        for quadrant_idx, quadrant in enumerate(background_quadrants[background]):
            # Decide the boundary of each background
            object_position_range= get_position_range(background_quadrants, background, scale=0.6)
            for obj_names in list(itertools.product(object_shapes, repeat=2)):
                for colors in color_pairs:
                    colors = [color_maps[color] for color in colors]
                    configs = []
                    
                    # Iterate over object names and create object configurations
                    for i, obj_name in enumerate(obj_names):
                        obj_range = object_size_ranges[obj_name]
                        size = random.uniform(min(obj_range), max(obj_range))
                        actual_size = actual_size * 1.1 / 2 if obj_name == "custom" else size
                        ranges = [tuple(range) for range in object_position_range]
                        config = create_config(obj_name, colors[i], actual_size, ranges)
                        configs.append(config)
                            
                    # register configures 
                    env.register_configures(collate_infos(configs)) 
                    _ = env.reset(options={"reconfigure": True})
                    env.initialize_objects(regular=True, background=background)                        

                    place_main_objects(env, configs)                          

                    save_dir = f"{final_dir}/object{idx}.png"
                    views = [f"front{background}", f"side{background}", f"top{background}"]
                    collect_and_save(env, save_dir, steps=1, mode=views)
           
                    output = {
                        'episode_id': idx,
                        "source": save_dir,
                        "obj_names": obj_names,
                        'sizes': [config['size'] for config in configs],
                        "colors": [config['color'] for config in configs],
                        "initiate_area": ranges,
                        "quadrant": quadrant_idx,
                        "background": background,
                    }
                    outputs.append(output)
                    idx += 1
                            
    
    write_out(f"{data_dir}/index.jsonl", outputs)

del env


[2024-10-15 06:08:03.398] [svulkan2] [error] GLFW error: X11: The DISPLAY environment variable is missing
[2024-10-15 06:08:03.398] [svulkan2] [warning] Continue without GLFW.
  0%|          | 0/4 [00:14<?, ?it/s]


KeyboardInterrupt: 